# Twitter Sentiment Analysis

# Problem Overview

# Data Understanding

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import TweetTokenizer
import string

import re

In [2]:
data = pd.read_csv('Data/crowdflower_tweet_data.csv', encoding = 'latin1')
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [3]:
data.shape

(9093, 3)

In [4]:
data.columns

Index(['tweet_text', 'emotion_in_tweet_is_directed_at',
       'is_there_an_emotion_directed_at_a_brand_or_product'],
      dtype='object')

In [5]:
data['emotion_in_tweet_is_directed_at'].value_counts(normalize=True)

iPad                               0.287451
Apple                              0.200851
iPad or iPhone App                 0.142814
Google                             0.130659
iPhone                             0.090246
Other Google product or service    0.089031
Android App                        0.024613
Android                            0.023701
Other Apple product or service     0.010635
Name: emotion_in_tweet_is_directed_at, dtype: float64

In [6]:
data['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts(normalize=True)

No emotion toward brand or product    0.592654
Positive emotion                      0.327505
Negative emotion                      0.062686
I can't tell                          0.017156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: float64

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [8]:
data.isna().sum()

tweet_text                                               1
emotion_in_tweet_is_directed_at                       5802
is_there_an_emotion_directed_at_a_brand_or_product       0
dtype: int64

In [9]:
data.duplicated().sum()

22

In [10]:
data[data.duplicated()].head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
468,"Before It Even Begins, Apple Wins #SXSW {link}",Apple,Positive emotion
776,Google to Launch Major New Social Network Call...,NaN,No emotion toward brand or product
2232,Marissa Mayer: Google Will Connect the Digital...,NaN,No emotion toward brand or product
2559,Counting down the days to #sxsw plus strong Ca...,Apple,Positive emotion
3950,Really enjoying the changes in Gowalla 3.0 for...,Android App,Positive emotion


# Data Cleaning


In [11]:
# Renaming columns
data.columns = ['Tweet', 'Product/Brand', 'Emotion']
data

,Tweet,Product/Brand,Emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [12]:
data = data.dropna(subset=['Tweet'])

In [13]:
data['Product/Brand'].fillna("Unknown Product", inplace = True)
data.isna().sum()

Tweet            0
Product/Brand    0
Emotion          0
dtype: int64

In [14]:
data.drop_duplicates(keep='first', inplace=True)

In [15]:
emotion = {'Positive emotion': 'Positive', 'Negative emotion': 'Negative', 
                'No emotion toward brand or product': 'Neutral', 
                "I can't tell": 'Unknown'}
data['Emotion'] = data['Emotion'].map(emotion)
data.head()

,Tweet,Product/Brand,Emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive


In [16]:
data['Emotion'].value_counts()

Neutral     5375
Positive    2970
Negative     569
Unknown      156
Name: Emotion, dtype: int64

In [17]:
pd.set_option("display.max_colwidth", 300)
data[data['Emotion']=='Unknown']


,Tweet,Product/Brand,Emotion
90,Thanks to @mention for publishing the news of @mention new medical Apps at the #sxswi conf. blog {link} #sxsw #sxswh,Unknown Product,Unknown
102,ÛÏ@mention &quot;Apple has opened a pop-up store in Austin so the nerds in town for #SXSW can get their new iPads. {link} #wow,Unknown Product,Unknown
237,"Just what America needs. RT @mention Google to Launch Major New Social Network Called Circles, Possibly Today {link} #sxsw",Unknown Product,Unknown
341,The queue at the Apple Store in Austin is FOUR blocks long. Crazy stuff! #sxsw,Unknown Product,Unknown
368,Hope it's better than wave RT @mention Buzz is: Google's previewing a social networking platform at #SXSW: {link},Unknown Product,Unknown
...,...,...,...
9020,It's funny watching a room full of people hold their iPad in the air to take a photo. Like a room full of tablets staring you down. #SXSW,Unknown Product,Unknown
9032,"@mention yeah, we have @mention , Google has nothing on us :) #SXSW",Unknown Product,Unknown
9037,"@mention Yes, the Google presentation was not exactly what I was expecting. #sxsw",Unknown Product,Unknown
9058,&quot;Do you know what Apple is really good at? Making you feel bad about your Xmas present!&quot; - Seth Meyers on iPad2 #sxsw #doyoureallyneedthat?,Unknown Product,Unknown


In [18]:
data = data[data['Emotion']!='Unknown']
data['Emotion'].value_counts()

Neutral     5375
Positive    2970
Negative     569
Name: Emotion, dtype: int64

In [19]:
# Create an instance of Regexptoenizer
tokenizer = TweetTokenizer("(?u)\w{3,}",
                          strip_handles=True)


#Create a list of stop words
stopwords_list = stopwords.words('english')

#Create an instance of the PorterStemmer
stemmer =PorterStemmer()


In [20]:
def preprocess_text(text, tokenizer,stopwords_list,stemmer):
    text = text.lower()
    token =tokenizer.tokenize(text)
    stop_words  = [word for word in token if word not in stopwords_list]
    stemmed_stopwords = [stemmer.stem(word) for word in stop_words]
    return stemmed_stopwords

In [21]:
text_data = data.Tweet.apply(lambda x: preprocess_text(x, tokenizer,stopwords_list,stemmer))
text_data

0                                                                                       [., 3g, iphon, ., 3, hr, tweet, #rise_austin, ,, dead, !, need, upgrad, ., plugin, station, #sxsw, .]
1                                                                                    [know, ?, awesom, ipad, /, iphon, app, like, appreci, design, ., also, ,, they'r, give, free, ts, #sxsw]
2                                                                                                                                                   [wait, #ipad, 2, also, ., sale, #sxsw, .]
3                                                                                                                                  [hope, year', festiv, crashi, year', iphon, app, ., #sxsw]
4                                              [great, stuff, fri, #sxsw, :, marissa, mayer, (, googl, ), ,, tim, o'reilli, (, tech, book, /, confer, ), &, matt, mullenweg, (, wordpress, )]
                                                  

In [22]:
data["preprocessed_text"] = text_data
data

,Tweet,Product/Brand,Emotion,preprocessed_text
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative,"[., 3g, iphon, ., 3, hr, tweet, #rise_austin, ,, dead, !, need, upgrad, ., plugin, station, #sxsw, .]"
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive,"[know, ?, awesom, ipad, /, iphon, app, like, appreci, design, ., also, ,, they'r, give, free, ts, #sxsw]"
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive,"[wait, #ipad, 2, also, ., sale, #sxsw, .]"
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative,"[hope, year', festiv, crashi, year', iphon, app, ., #sxsw]"
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive,"[great, stuff, fri, #sxsw, :, marissa, mayer, (, googl, ), ,, tim, o'reilli, (, tech, book, /, confer, ), &, matt, mullenweg, (, wordpress, )]"
...,...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive,"[ipad, everywher, ., #sxsw, {, link, }]"
9089,"Wave, buzz... RT @mention We interrupt your regularly scheduled #sxsw geek programming with big news {link} #google #circles",Unknown Product,Neutral,"[wave, ,, buzz, ..., rt, interrupt, regularli, schedul, #sxsw, geek, program, big, news, {, link, }, #googl, #circl]"
9090,"Google's Zeiger, a physician never reported potential AE. Yet FDA relies on physicians. &quot;We're operating w/out data.&quot; #sxsw #health2dev",Unknown Product,Neutral,"[google', zeiger, ,, physician, never, report, potenti, ae, ., yet, fda, reli, physician, ., "", we'r, oper, w, /, data, ., "", #sxsw, #health2dev]"
9091,Some Verizon iPhone customers complained their time fell back an hour this weekend. Of course they were the New Yorkers who attended #SXSW.,Unknown Product,Neutral,"[verizon, iphon, custom, complain, time, fell, back, hour, weekend, ., cours, new, yorker, attend, #sxsw, .]"


# Exploratory Data Analysis
---

# Modelling

# Conclusion